In [1]:
from utils.merge_model_objects import merge_instances

In [2]:
for bl in Byline.objects.all():
    clones = Byline.objects.filter(story=bl.story, contributor=bl.contributor)
    if len(clones) > 1:
        merge_instances(*clones)

In [161]:
for image in ImageFile.objects.order_by('pk'):
    clones = ImageFile.objects.filter(pk__gte=image.pk, _imagehash=image._imagehash)
    if clones.count() > 1:
        clones = clones.order_by('-created', '-full_width')
        print()
        for im in clones:
            print(f'{im.pk:>5} {str(im)[:50]:<50} {im._imagehash}  {im.created:%d/%m %X}  {im._size:>10}')


43358 16-ANM-larsvaular-HKH-09_6vOB48L.jpg               070f0f1b0f874f4d  14/08 17:19:15       94884
43303 36144839090-821d81afae-o_Jb4xtfU.jpg               070f0f1b0f874f4d  13/08 15:14:52    14995601

43560 16-ANM-oyareport-AAGS-06-OciBQws.jpg               6654a3a2935b3ba2  16/08 14:36:22      123536
43429 16-ANM-oyareport-AAGS-06_mV6mzN5.jpg               6654a3a2935b3ba2  14/08 17:21:33      127683

44116 21392833-10154706933586502-2105708741-o_Q4qBgkG.jp 859925366f2f2d35  05/09 22:30:44      219267
43879 18-NYH-Sveinung-Rotvatn-AS-08_akWumZT.jpg          859925366f2f2d35  29/08 15:18:03      180352

43966 29070079203-ab38c11542-o-teqdT94.jpg               cf8b07ce219327a5  30/08 15:54:37    16620554
43932 29070079203-ab38c11542-o_DqUyTIv.jpg               cf8b07ce219327a5  29/08 18:39:41       76100

44247 36797860591-fc93cf2176-o_tMwASWH.jpg               e35631b69692244c  13/09 13:01:28     7850156
44076 19-MAG-ungdomspolitikk-OA-21_9G8OpXM.jpg           e35631b69692244c  04

In [80]:
for im in (image, clones[0]):
    print(im, im.md5, im.size, im.full_height, im.full_width, im._imagehash)

36144839090-821d81afae-o_Jb4xtfU.jpg ef61c30647e57092e50a8bc1c34a7067 14995601 1001 1500 070f0f1b0f874f4d
16-ANM-larsvaular-HKH-09_6vOB48L.jpg ef6c672fe28e3396a1bb1f4753706c0c 94884 1001 1500 070f0f1b0f874f4d


In [113]:
from PIL import Image

imgs = [Image.open(im.source_file.file) for im in ImageFile.objects.filter(_imagehash='cf8b07ce219327a5')]

In [124]:
from pprint import pprint
for img in imgs:
    print(img._getexif().get(315))

Susanne Alexandra Tengs
Susanne Alexandra Tengs


In [3]:
for img in ImageFile.objects.all():
    img.add_exif_from_file()
    print(img.exif_data)

{'ResolutionUnit': '2', 'ExifOffset': '276', 'Make': 'Canon', 'Model': 'Canon EOS 5D Mark III', 'Software': 'Adobe Photoshop CC 2017 (Macintosh)', 'Orientation': '1', 'DateTime': '2017:02:06 08:44:54', 'Artist': 'Adrian Nielsen', 'Copyright': 'Adrian Nielsen', 'XResolution': ['300', '1'], 'YResolution': ['300', '1'], 'ExifVersion': '0230', 'ShutterSpeedValue': ['7643856', '1000000'], 'ApertureValue': ['3356144', '1000000'], 'DateTimeOriginal': '2017:02:06 09:21:20', 'DateTimeDigitized': '2017:02:06 09:21:20', 'ExposureBiasValue': ['0', '1'], 'MaxApertureValue': ['3', '1'], 'MeteringMode': '5', 'ColorSpace': '65535', 'Flash': '16', 'FocalLength': ['200', '1'], 'ExifImageWidth': '2000', 'ExifImageHeight': '1333', 'SceneCaptureType': '0', 'FocalPlaneXResolution': ['52428800', '32768'], 'FocalPlaneYResolution': ['52428800', '32768'], 'SubsecTime': '00', 'SubsecTimeOriginal': '00', 'SubsecTimeDigitized': '00', 'FocalPlaneResolutionUnit': '3', 'ExposureTime': ['1', '200'], 'FNumber': ['32', 

AttributeError: 'NoneType' object has no attribute 'items'

In [28]:
folks = Contributor.objects.filter(display_name__startswith='Kristina')
for kris in folks:
    print(kris)
    print(kris.byline_set.all())
    o
    
    
merge_instances(*list(folks))

Kristina Holt
<QuerySet [<Byline: @bl: Tekst: Kristina Holt>]>
Kristina Kvammen
<QuerySet [<Byline: @bl: Tekst: Kristina Kvammen, Magasinfotosjef>, <Byline: @bl: Foto: Kristina Kvammen>]>
> <ipython-input-27-92d0f756f523>(36)merge_instances()
     34     # primary object
     35     set_trace()
---> 36     for alias_object in alias_objects:
     37         # Migrate all foreign key references from alias object to primary
     38         # object.

ipdb> fields
*** NameError: name 'fields' is not defined
ipdb> relations
(<ManyToManyRel: stories.story>, <ManyToOneRel: stories.byline>, <ManyToOneRel: photo.imagefile>, <ManyToOneRel: contributors.stint>)
ipdb> q


BdbQuit: 

'aGVsbG8gd29ybGQ='

In [21]:
for person in Contributor.objects.order_by('display_name'):
    print(person.pk, person)

14854 1997. 20. Mantel
14199 Adrian Nielsen
14398 Adrian Simen Holm
14636 Alf Grønli Simensen
14610 Angelique Culvin
14723 Arvid Folke Järnbert
14801 Åsmund M. Kvifte
14799 Aurora Sæverud
14468 Birk Tjeldflaat Helle
14868 Camilla Alexandra Lie
14862 Eirin Bruholt
14665 Emilie Solberg
14391 Erlend Daae
12404 Fredrik Arff
14859 Frida Fliflet
14654 Gard Oterholm
14803 Gin&Tonic
13257 Gro Bjørnerud Mo
13419 Håken Lid
14681 Hannah Berg
14677 Hanna Hjardar
14327 Hanna Skotheim
14497 Hans Christian Paulsen
14648 Håvard Røsæg
14426 Heidi Bang
14569 Henriette Dæhli
14711 Henrik Giæver
14701 Indigo Trigg-Hauger
14579 Ingeborg Grindheim Slinde
14764 Ingrid Ekeberg
14742 Ingrid N. Ekeberg
14592 Jens Lægreid
14858 Johannes Fjeld
14853 Julie Brundtland
14656 Julie Brundtland Løvseth
14797 Kari Eiring
14644 Kenneth Haug
14481 Knut Arne Oseid
14861 Kristian Müller Andersen
14432 Kristina Holt
14466 Kristina Kvammen
14758 Kristine Sørheim
14442 Lina Christensen
14555 Louisa Boulaziz
14855 Lucas Ibanez-

In [163]:
from collections import Counter
def duplicates(queryset, attr, order_by=()):
    counter = Counter(queryset.values_list(attr, flat=True))
    duped_values = [el for el in counter if counter[el] > 1]
    sets = []
    for value in duped_values:
        items = queryset.filter(**{attr: value}).order_by(*order_by)
        sets.append(items)
    return sets

images = ImageFile.objects.exclude(_md5=None)
md5_dupes = duplicates(images, '_md5', ['created'])
imhash_dupes = duplicates(images, '_imagehash', ['created'])

def print_dupes(dupe_images, attr):
    print(f'{attr}-dupes: {len(dupe_images)}')
    for qs in dupe_images:
        print()
        for img in qs:
            print(getattr(img, attr), img)

print_dupes(md5_dupes, '_md5')
print_dupes(imhash_dupes, '_imagehash')

_md5-dupes: 0
_imagehash-dupes: 5

070f0f1b0f874f4d 36144839090-821d81afae-o_Jb4xtfU.jpg
070f0f1b0f874f4d 16-ANM-larsvaular-HKH-09_6vOB48L.jpg

6654a3a2935b3ba2 16-ANM-oyareport-AAGS-06_mV6mzN5.jpg
6654a3a2935b3ba2 16-ANM-oyareport-AAGS-06-OciBQws.jpg

859925366f2f2d35 18-NYH-Sveinung-Rotvatn-AS-08_akWumZT.jpg
859925366f2f2d35 21392833-10154706933586502-2105708741-o_Q4qBgkG.jpg

cf8b07ce219327a5 29070079203-ab38c11542-o_DqUyTIv.jpg
cf8b07ce219327a5 29070079203-ab38c11542-o-teqdT94.jpg

e35631b69692244c 19-MAG-ungdomspolitikk-OA-21_9G8OpXM.jpg
e35631b69692244c 36797860591-fc93cf2176-o_tMwASWH.jpg


## Merge duplicates

In [47]:
def assert_equal(objects, attrs=()):
    master, *others = objects
    for other in others:
        for attr in attrs:
            master_attr = getattr(master, attr)
            other_attr = getattr(other, attr)
            if master_attr != other_attr:
                raise ValueError(f'{master}.{attr}={master_attr}\n{other}.{attr}={other_attr}')
    
def merge_imghash_dupes():
    images = ImageFile.objects.exclude(_imagehash='')
    dupes = duplicates(images, '_imagehash', ['-full_width','-_size','created'])
    c = len(dupes)
    for qs in dupes:
        items = list(qs)
        #try: 
        #    assert_equal(items, attrs=['original', '_imagehash'])
        #except ValueError as e:
        #    print(e)
        #    continue
        merge_instances(*items)
        print(c)
        c -= 1
        for item in items:
            print(f'{item._imagehash} {item.size} {item.md5} {item.full_width} {item}')

merge_imghash_dupes()
            
            

121
692131313327cb8d 118062 70960096b0203cd38a8a6b1d3395bb1e 1500 04-KUL-bareegil-SN-4.jpg
692131313327cb8d 117250 a06f087ce4619867035bc0201a97db68 1500 ARA6564.jpg
120
a48472b47262220c 193433 5d92ce158af8ac0f2faace772a7676ae 1500 04-KUL-bareegil-SN-1.jpg
a48472b47262220c 192620 1436eabbf9464e3e025074075d302500 1500 ARA6525.jpg
119
a9d953c929626373 428540 e01122d4cff3d397aa39bcc4a8b3ebef 1500 05-DEB-forskerintervju-EBJ-9.jpg
a9d953c929626373 427284 16c0dc7afa4eb15c167725123a58e5ef 1500 DSCF0298-copy.jpg
118
2ccde6a3b4c7b631 147374 7996719498ef20975cadc66bd8ba6987 1500 05-DEB-forskerintervju-EBJ-7.jpg
2ccde6a3b4c7b631 146117 0bd4c6f0a332486faaf9449a848abbad 1500 DSCF0301.jpg
117
e8d9d3c9297263fb 416410 4a4e646c7cade894366c6a0e27fae318 1500 05-DEB-forskerintervju-EBJ-6.jpg
e8d9d3c9297263fb 415146 cc0fe9acebf77928a198ab8fcba05290 1500 DSCF0298.jpg
116
5a2cece373365afb 232635 e8b1ade817e8d606cc7528c85e36cb62 1500 05-DEB-forskerintervju-EBJ-2.jpg
5a2cece373365afb 231371 3c059447851ae9d6b57a

73
4f0d867e1b9b0607 171286 3567d6c5a76f55abff99fc8b180d4f7b 1500 JH77029.jpg
4f0d867e1b9b0607 166140 3b584d44b683d91e6c503a4f6f9d848f 1500 07-KUL-skitur-GO-18.jpg
72
63b243238bedec16 299718 b5256c1c734e0d2f0b41e5c98e4dd204 1500 07-KUL-skitur-GO-16.jpg
63b243238bedec16 298541 fad0a056146efcd95f2430f66a1f801d 1500 JH76951.jpg
71
242c3aaba3d52b13 232743 94a189a47e3c4ab62418cec2a07d2625 1500 07-KUL-skitur-GO-12.jpg
242c3aaba3d52b13 231566 6b9f5ef71fb969fd28f6e90084d27172 1500 JH76914.jpg
70
360f172f064f2533 128094 70331f0d9b62dfec93807ef3364d506b 1500 JH77180.jpg
360f172f064f2533 122947 751cfb813346c489979a2d71602f5bb4 1500 07-KUL-skitur-GO-20.jpg
69
8d4c5d5c6614a4a6 278335 faa74c7c6339479a15b8af2d0f8bc0be 1500 JH76841.jpg
8d4c5d5c6614a4a6 273188 12da16625495dbd4ff1e7e4552cf8bf4 1500 07-KUL-skitur-GO-06.jpg
68
c71931b3f1ada95e 176025 d3c7f2446afa73ec3052c4502fa1853d 1500 07-NYH-toksikologi-AN-06.jpg
c71931b3f1ada95e 174731 341d7d203bba923a9793c2ff2b106b7a 1500 07-NYH-Toksikologi-AN-6.jpg
6

27
43d9d9b9a56d3a26 45806 8a01737c8cb2b63c9185808c79a0a6cf 1024 Illustrasjon-kronikk-Karlsoen-foto-kjetil-blom.jpg
43d9d9b9a56d3a26 44982 69eed8d4e3158b473227169571e908ff 1024 ole-p-ottersen-02.jpg
26
afa6a56d4dc66265 735229 0dcb6885642c65e5295aa486d3651e1d 2048 netturix09.jpg
afa6a56d4dc66265 175039 d97b461c471e2dac651c9aa33a1ac90f 1500 netturix09.jpg
25
4f6727676fcfc795 51482 9f0471ea69392afa62b5bb125ddbc4d3 1024 30-MAG-repdialektfo-11-HE.jpg
4f6727676fcfc795 51180 c5939fdac959667f376a84cc02028ef6 1024 30-MAG-repdialektfo-06-HE.jpg
24
f4dad2614a634969 85426 082d05943c3059a4b2a90e10d52857d4 900 21-arbeidsmengde-paa-vethoeyskolen-01-RRJ.jpg
f4dad2614a634969 85305 5277df35be31bcfe2fc0e829c0681cfd 900 32-veterinAErhoeyskolen-Kristoffer-Forbergarkivfoto-01-RRJ.jpg
23
aaa262b270691a40 49313 34e2f9fecfd5d9412b11de5167339a21 838 31-adnotam-Rugaas.jpg
aaa262b270691a40 74260 57fdf961c97e3dd0bc01c30b12b7a1f5 503 14-adnotam-Rugaas-Web.jpg
22
040b3a32191e86c6 1394852 b26afc254919dd8c8b62b3bb34125

In [165]:
#finne personer som kan merges

from fuzzywuzzy import process
from django.db.models import Count
contributors = Contributor.objects.annotate(
        num_bylines=Count('byline')
    ).order_by('-num_bylines')

#one_hit_wonders = contributors.filter(num_bylines=1)
one_hit_wonders = []
with open('one_hit_wonders.txt') as fp:
    for line in fp.readlines():
        id = int(line.split()[0])
        one_hit_wonders.append(id)
        
    
print(one_hit_wonders)

[12752, 13893, 13147, 13331, 12737, 12932, 12423, 13729, 13745, 12648, 14594, 13635, 13182, 12749, 14292, 13799, 13235, 12821, 12989, 13262, 13020, 13281, 13739, 12741, 13112, 13612, 13624, 12893, 12591, 13494, 13136, 13176, 13153, 12827, 12432, 13610, 12551, 13554, 13833, 12405, 12568, 13495, 13560, 13118, 12550, 13139, 12909, 13464, 13187, 14308, 14631, 13941, 13126, 13410, 12716, 13069, 14396, 13659, 12686, 13744, 12938, 12531, 12769, 13802, 12505, 14088, 13047, 12541, 13643, 13923, 13529, 12859, 14002, 13276, 12892, 13291, 12468, 12546, 13578, 13155, 14074, 12905, 12604, 13053, 13707, 13043, 13874, 13065, 12833, 13062, 12969, 13480, 12513, 13272, 13057, 13019, 13263, 12899, 12510, 13517, 13860, 12674, 12491, 13561, 12421, 13298, 14674, 13753, 13067, 13453, 13750, 13455, 13034, 12707, 12554, 13427, 13089, 13258, 12509, 13111, 13459, 13734, 13875, 12923, 13134, 12983, 12494, 13083, 14028, 12810, 13730, 13971, 13465, 13366, 12740, 13378, 12977, 13959, 14269, 13593, 12544, 12387, 12594

In [166]:
nn = Contributor.objects.get(display_name="Nomen Nescio")
for bl in nn.byline_set.all():
    bylines = bl.story.byline_set.all()
    for byline in bylines:
        print(byline) 


@bl: Av: Nomen Nescio, Svar: 1. Granada 2. Granateple 3. Cava 4. Marokko 5. Morgenbladet 6. Stiftsgården 7. Eric Heiden 8. Komponist 9. Trøfler 10. Bo 11. 1968 og 1963 12. Jack Ruby 13. Grønland 14. Isabell Allende 15. 1962
@bl: Av: 1997. 20. Mantel
@bl: Av: Nomen Nescio, text: Hans Christian Paulsen foto Lucas Leonardo Ibanez-Fæhn
@bl: Tekst: Henriette Dæhli


In [ ]:
import requests
import concurrent.futures

URLS = [im.source_file.url for im in ImageFile.objects.all()]

URLS = [im.source_file.url for im in ImageFile.objects.all()]
def get_http_status(url, timeout=10):
    return requests.head(url, timeout = 5).status_code

# We can use a with statement to ensure threads are cleaned up promptly
broken_urls = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(get_http_status, url): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            status_code = future.result()
            if status_code == 404:
                broken_urls.append(url)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))


In [134]:
from pathlib import Path
import imagehash
import PIL
import requests

<ImageFileQuerySet []>

In [224]:
# Make searchspace
images = ImageFile.objects.all()
vals = images.values( '_imagehash', 'id' )
haystack = [(v['id'], imagehash.hex_to_hash(v['_imagehash'])) for v in vals]
len(images)

330

In [225]:
def closest_image(_imagehash, n=1, cutoff=200, haystack=haystack):
    candidates = [(_imagehash - h, pk, h) for pk, h in haystack if _imagehash - h < cutoff]
    return [(d, ImageFile.objects.get(pk=pk)) for d, pk, h in sorted(candidates)[:n]]

In [262]:
from IPython.display import HTML
def display(images):
    html = ''
    for image in images:
        title = f'src:  {image.original}\nhash: {image.imagehash}\nmd5:  {image.md5}\nsize: {image.size}'
        html += f'<div style="width: calc(50% - 10px); display: inline-block; padding: 5px" >'
        html += f'<pre style="position: relative; background: rgba(255, 255, 255, 0.8)">{title}</pre>'
        html += f'<img title="{title}" src="http://localhost:8000{image.large.url}" />'
        html += f'</div>'
        
    return HTML(html)

In [266]:
#img = ImageFile.objects.order_by('?').first()
#img = ImageFile.objects.filter(_imagehash='e35631b69692244c')[0]
#print(img._imagehash)
disp = []
found = set()
for img in ImageFile.objects.all():
    matches = closest_image(imagehash.hex_to_hash(img._imagehash), 2, 10)
    if len(matches) > 1:
        pks = [m[1] for m in matches]
        if not any(pk in found for pk in pks):
            disp += pks
            found.update(pks)
            
        
display(disp)